In [1]:
import numpy as np
import pandas as pd
import glob
import os

dirs = glob.glob('input/dataset/train/*')
images = list()
labels = list()
for d in dirs:
    images += glob.glob(d + '/*')
    labels += [int(os.path.basename(d))] * len(glob.glob(d + '/*'))
    
from sklearn.utils import shuffle
df = pd.DataFrame({'images': images, 'labels': labels})
df = shuffle(df)
df.head()

,images,labels
44500,input/dataset/train/9/n04615644_2314_0.jpg,9
44055,input/dataset/train/9/n04222470_6590_0.jpg,9
20476,input/dataset/train/4/n03384891_14147_0.jpg,4
45793,input/dataset/train/6/n03010795_330_0.jpg,6
29069,input/dataset/train/8/n03013438_19470_0.jpg,8


In [2]:
df.shape

(62258, 2)

In [3]:
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization
from keras.models import Model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

base_model = VGG19(weights='imagenet', include_top=False)
    
model = Model(inputs=base_model.input, outputs=base_model.output)

/home/winter/miniconda3/envs/dl/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/winter/miniconda3/envs/dl/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
from tqdm import tqdm
import pickle as pkl

count = 0
for img_path, label in zip(df.images, df['labels']):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = preprocess_input(x)
    features = model.predict(np.expand_dims(x, axis=0))
    features = np.squeeze(features)

    y = np.zeros(15)
    y[label] = 1
    
    name = os.path.basename(img_path).split()[0]
    with open('input/dataset/train_features/{}.pkl'.format(name), 'wb') as file:
        pkl.dump([features, y], file)
        
    count += 1
    if count % 3000 == 0:
        print(count)

3000
6000
9000
12000
15000
18000
21000
24000
27000
30000
33000
36000
39000
42000
45000
48000
51000
54000
57000
60000


In [5]:
with open('input/dataset/train_features/{}.pkl'.format(name), 'rb') as file:
    ip = pkl.load(file)

ip[0].shape, ip[1].shape

((7, 7, 512), (15,))

In [6]:
from tqdm import tqdm
import pickle as pkl

test_images = os.listdir('input/dataset/test/')

for img_path in tqdm(test_images):
    img_path = os.path.join('input/dataset/test/', img_path)
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = preprocess_input(x)
    features = model.predict(np.expand_dims(x, axis=0))
    
    name = os.path.basename(img_path).split()[0]
    with open('input/dataset/test_features/{}.pkl'.format(name), 'wb') as file:
        pkl.dump(features, file)

100%|██████████| 21273/21273 [04:20<00:00, 81.82it/s]
